In [20]:
import os
os.chdir('/home/tapas/Downloads/Brainwaves/')

In [21]:
os.listdir(".")

['braiwaves.R',
 'test.csv',
 'train.csv',
 'sample_submission.csv',
 'all_data.csv']

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [212]:
train = pd.read_csv('train.csv')

In [213]:
y = train['return']

In [202]:
train_len = len(train)

In [171]:
test = pd.read_csv('test.csv')

In [ ]:
protfolio.ID = test['portfolio_id'] 

In [172]:
all_data = pd.concat((train.loc[:,'pf_category':'status'],
                      test.loc[:,'pf_category':'status']))

In [173]:
all_data.head(5)

,pf_category,start_date,sold,country_code,euribor_rate,currency,libor_rate,bought,creation_date,indicator_code,sell_date,type,hedge_value,status
0,B,20040720,110000000.0,T,0.02074,USD,2.332216,1.098097e+08,20040720,NaN,20040812,B,NaN,NaN
1,A,20040709,176671000.0,N,0.02074,GBP,5.269617,1.760084e+08,20040723,NaN,20040812,C,NaN,NaN
2,A,20040723,56474000.0,T,0.02074,USD,2.332216,5.637953e+07,20040723,NaN,20040817,A,NaN,NaN
3,A,20040609,164813000.0,T,0.02074,USD,2.332216,1.645088e+08,20040723,NaN,20040713,A,NaN,NaN
4,B,20040609,140800000.0,T,0.02074,USD,2.332216,1.405402e+08,20040723,NaN,20040713,B,NaN,NaN


In [174]:
all_data.tail(5)

,pf_category,start_date,sold,country_code,euribor_rate,currency,libor_rate,bought,creation_date,indicator_code,sell_date,type,hedge_value,status
4796,C,20160621,55000000.0,M,-0.00349,CHF,-0.556873,5.499519e+07,20160621,True,20160628,E,False,True
4797,C,20160621,138809000.0,M,-0.00349,CHF,-0.556873,1.387969e+08,20160621,True,20160628,E,False,True
4798,B,20160622,28732000.0,N,-0.00349,GBP,0.960652,2.872930e+07,20160622,True,20160629,C,False,True
4799,C,20160623,34419000.0,M,-0.00349,CHF,-0.556873,3.441599e+07,20160623,True,20160630,E,False,True
4800,C,20160628,53713000.0,M,-0.00349,CHF,-0.556873,5.370778e+07,20160628,True,20160705,E,False,True


In [175]:
print("all_data size is : {}".format(all_data.shape))

all_data size is : (14167, 14)


# Deal with Missing Values

In [176]:
all_data['bought'].fillna(0, inplace=True)
all_data['sold'].fillna(0, inplace=True)

In [177]:
na_totals = all_data.isnull().sum().sort_values(ascending=False)
na_totals[na_totals>0]

hedge_value       8552
indicator_code    8550
status            4541
libor_rate         739
dtype: int64

In [178]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio
hedge_value,60.365638
indicator_code,60.351521
status,32.053363
libor_rate,5.216348


In [179]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index, y=all_data_na)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

In [180]:
all_data["hedge_value"].fillna(0,inplace = True)

In [181]:
all_data["indicator_code"].fillna(0,inplace = True)

In [182]:
#all_data["desk_id"] = all_data["desk_id"].fillna("None")

In [183]:
all_data["status"].fillna(0,inplace = True)

In [184]:
#all_data['status'] = all_data['status'].fillna(all_data['status'].mode()[0])

In [185]:
all_data["libor_rate"] = all_data["libor_rate"].transform(lambda x: x.fillna(x.median()))

In [186]:
all_data.isnull().sum()

pf_category       0
start_date        0
sold              0
country_code      0
euribor_rate      0
currency          0
libor_rate        0
bought            0
creation_date     0
indicator_code    0
sell_date         0
type              0
hedge_value       0
status            0
dtype: int64

# Feature Analysis

In [187]:
all_data.head(5)

,pf_category,start_date,sold,country_code,euribor_rate,currency,libor_rate,bought,creation_date,indicator_code,sell_date,type,hedge_value,status
0,B,20040720,110000000.0,T,0.02074,USD,2.332216,1.098097e+08,20040720,0,20040812,B,0,0
1,A,20040709,176671000.0,N,0.02074,GBP,5.269617,1.760084e+08,20040723,0,20040812,C,0,0
2,A,20040723,56474000.0,T,0.02074,USD,2.332216,5.637953e+07,20040723,0,20040817,A,0,0
3,A,20040609,164813000.0,T,0.02074,USD,2.332216,1.645088e+08,20040723,0,20040713,A,0,0
4,B,20040609,140800000.0,T,0.02074,USD,2.332216,1.405402e+08,20040723,0,20040713,B,0,0


In [188]:
# Explore Fare distribution 
g = sns.distplot(all_data["euribor_rate"], color="m", label="Skewness : %.2f"%(all_data["euribor_rate"].skew()))
g.legend(loc="best")

In [189]:
all_data["euribor_rate"] = all_data["euribor_rate"].map(lambda i: np.log(i) if i > 0 else 0)

In [190]:
all_data["libor_rate"] = all_data["libor_rate"].map(lambda i: np.log(i) if i > 0 else 0)

In [191]:
all_data["sold"] = all_data["sold"].map(lambda i: np.log(i) if i > 0 else 0)

In [192]:
all_data["bought"] = all_data["bought"].map(lambda i: np.log(i) if i > 0 else 0)

In [193]:
all_data.head(2)

,pf_category,start_date,sold,country_code,euribor_rate,currency,libor_rate,bought,creation_date,indicator_code,sell_date,type,hedge_value,status
0,B,20040720,18.515991,T,-3.875691,USD,0.846819,18.514259,20040720,0,20040812,B,0,0
1,A,20040709,18.989800,N,-3.875691,GBP,1.661958,18.986042,20040723,0,20040812,C,0,0


In [194]:
from sklearn.preprocessing import LabelEncoder

In [195]:
le = LabelEncoder()

In [196]:
for col in all_data.columns:
    if col in ['pf_category','country_code','currency','indicator_code','type','status']:
        all_data[col]= le.fit_transform(all_data[col])

In [197]:
for col in all_data.columns:
    if col in ['pf_category','country_code','currency','indicator_code','type','status','hedge_value']:
        all_data[col]= all_data[col].astype('category')

In [198]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14167 entries, 0 to 4800
Data columns (total 14 columns):
pf_category       14167 non-null category
start_date        14167 non-null int64
sold              14167 non-null float64
country_code      14167 non-null category
euribor_rate      14167 non-null float64
currency          14167 non-null category
libor_rate        14167 non-null float64
bought            14167 non-null float64
creation_date     14167 non-null int64
indicator_code    14167 non-null category
sell_date         14167 non-null int64
type              14167 non-null category
hedge_value       14167 non-null category
status            14167 non-null category
dtypes: category(7), float64(4), int64(3)
memory usage: 1.3 MB


In [200]:
for col in all_data.columns:
    if col in ['pf_category','country_code','currency','indicator_code','type','status','hedge_value']:
        all_data = pd.get_dummies(all_data, columns = [col],prefix="new")

In [210]:
X_train = all_data[:train_len]
test = all_data[train_len:]
y = train['return']
test.drop(labels=["return"],axis = 1,inplace=True)

KeyError: 'return'

# Model

In [207]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [208]:
model_ridge = Ridge()

In [209]:
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]

NameError: name 'y' is not defined

In [ ]:
cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation")
plt.xlabel("alpha")
plt.ylabel("rmse")

In [ ]:
cv_ridge.min()

In [ ]:
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(X_train, y)

In [ ]:
rmse_cv(model_lasso).mean()

In [ ]:
coef = pd.Series(model_lasso.coef_, index = X_train.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = pd.concat([coef.sort_values().head(10),
                     coef.sort_values().tail(10)])

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model")

In [ ]:
#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":model_lasso.predict(X_train), "true":y})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")

In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(X_train, label = y)
dtest = xgb.DMatrix(X_test)

params = {"max_depth":2, "eta":0.1}
model = xgb.cv(params, dtrain,  num_boost_round=500, early_stopping_rounds=100)

In [ ]:
model.loc[30:,["test-rmse-mean", "train-rmse-mean"]].plot()

In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1) #the params were tuned using xgb.cv
model_xgb.fit(X_train, y)

In [ ]:
xgb_preds = np.expm1(model_xgb.predict(X_test))
lasso_preds = np.expm1(model_lasso.predict(X_test))

In [ ]:
predictions = pd.DataFrame({"xgb":xgb_preds, "lasso":lasso_preds})
predictions.plot(x = "xgb", y = "lasso", kind = "scatter")

In [ ]:
preds = 0.7*lasso_preds + 0.3*xgb_preds

In [ ]:
solution = pd.DataFrame({"portfolio_id":protfolio.ID, "return":preds})
solution.to_csv("ridge_sol.csv", index = False)